In [415]:
import multiprocessing
import numpy as np
from joblib import Parallel, delayed
from ipynb.fs.defs.Fractool_Current import FracTool
import pickle

#open saved expected Hurst array and signal array
with open("Hurst_expected_pickle.txt", "rb") as fp:
    Hurst_expected = pickle.load(fp)
with open("signal_pickle.txt", "rb") as fp:
    raw_sig = pickle.load(fp)
raw_sig = np.asarray(raw_sig) #convert to numpy array
n = len(Hurst_expected) #number of signals
output = range(n) #initialize results array
#generate expected class array
Class_expected = np.tile([2,0], n) #since signals are alternating fGm/fBm, class will alternate between 1 and 2
#initialize empty arrays for actual Hurst and class results from FracTool
Hurst_actual = np.zeros(n) 
Class_actual = np.zeros(n)
misclassified_total = 0
diff = 0
summation = 0
def MAE_calc(i):
    '''This function returns the MAE and miscalculation rate (MR) of FracTool, with comparision to known Hurst values 
    of the generated signal array of both fbm and fgn signals'''
    global misclassified_total #allows variables to exist in function 
    global diff
    result = FracTool(raw_sig[i]) #run Fractool on each signal in signal array
    Hurst_actual[i] = result[1] #result[1] of FracTool is Hurst value
    Class_actual[i] = result[0] #result[0] of FracTool is Class
    if Class_actual[i] != Class_expected[i]: #if signal is misclassified
        misclassified_total = 1 #increase misclassified total by 1
        Hurst_expected[i] = 0 #set Hurst expected and actual to 0 - excludes them from MAE calculation
        Hurst_actual[i] = 0
    diff = abs(Hurst_actual[i] - Hurst_expected[i])
    return [misclassified_total, diff]

#multiprocessing for loop
num_cores = multiprocessing.cpu_count()
output = Parallel(n_jobs=num_cores)(delayed(MAE_calc)(i) for i in output)
#output is array of the signal array size with two columns
#column one is 0 or 1 (not misclassified or misclassified) and column two is the difference between the signal's
#expected Hurst and the actual Hurst FracTool generated
output = np.array(output)
#print(output)
misclassified_total = np.sum(output[:,0]) #sum of column one is the total number of signals misclassified
summation = np.sum(output[:,1]) #sum of column 2 is the total difference of all signals
MAE = summation/n #MAE is summation divided by total number of signals
MR = misclassified_total/n #MR is total number of misclassified signals divided by number of signals
print(MAE, MR)

[[0.         0.08906255]
 [0.         0.0584811 ]
 [0.         0.00741019]
 [0.         0.06669911]
 [0.         0.13700387]
 [0.         0.06411344]
 [0.         0.12848373]
 [0.         0.03775084]
 [0.         0.0857392 ]
 [0.         0.10428196]]
0.07790259790922147 0.0


In [405]:
#generate signal array
import pickle
from fbm import fbm, fgn, times
import random
from joblib import Parallel, delayed


n = 10 #n is number of signals  
length = 1000 #length is the length of each signal
#initialize arrays to iterate through
signals = range(n)
Hurst_expected = np.zeros(n)
#generate array of expected Hurst values 
for x in range(n):
     Hurst_expected[x] = random.random() #generate values between 0 and 1
        
def generate_signal(j):
    '''This function generates an array of signals, alternating between fBm and fGn signals'''
    Hurst_num = Hurst_expected[j] #Hurst value used is indexed from known Hurst array
    if j % 2 == 0: #every even index is a fBm signal
        sig = fbm(n=length+1, hurst = Hurst_num, length=1, method='daviesharte')[1:length+1]
    if j % 2 != 0: #every odd index is a fGn signal 
        sig = fgn(n=length, hurst = Hurst_num, length=1, method='daviesharte')    
    return sig

#multiprocessor for loop 
num_cores = multiprocessing.cpu_count()
signals = Parallel(n_jobs=num_cores)(delayed(generate_signal)(j)for j in signals)

#save/pickle signal array 
with open("signal_pickle.txt", "wb") as fp:
    pickle.dump(signals, fp)
#save/pickle Hurst expected array
with open('Hurst_expected_pickle.txt', "wb") as fp:
    pickle.dump(Hurst_expected,fp)
#save signal array as txt file for future reference    
with open('signal_copy.txt', 'w') as f:
    for signal in signals:
        f.write("%s\n" % signal)
#save Hurst expected array as txt file for future reference        
with open('Hurst_expected_copy.txt', 'w') as f:
    for Hurst in Hurst_expected:
        f.write("%s\n" % Hurst)


[array([-1.84620642e-03, -2.12794806e-03, -3.94323055e-03, -2.74965956e-03,
       -6.41735837e-03, -1.39376003e-02, -1.99438318e-02, -1.69438861e-02,
       -1.30324503e-02, -1.18922623e-02, -1.01413200e-02, -8.01815810e-03,
       -8.11229374e-03, -1.71082823e-03, -6.43249162e-03, -8.51668330e-03,
       -1.09597713e-02, -4.34178884e-03, -3.52024569e-03, -5.73838299e-03,
       -7.73982845e-03, -7.44737866e-03, -9.40239932e-03, -1.01290919e-02,
       -1.50548082e-02, -1.22687585e-02, -1.49667882e-02, -1.48762144e-02,
       -1.42533296e-02, -5.17791126e-03, -1.62268794e-03, -1.99744585e-04,
       -5.73538752e-04, -1.33450343e-03,  1.82195879e-03,  2.19973389e-03,
        4.32015693e-03,  7.37606729e-03,  1.17371665e-02,  1.63576109e-02,
        1.39312231e-02,  1.70380115e-02,  1.81605719e-02,  1.48078752e-02,
        9.01730127e-03,  1.14024875e-02,  1.05667556e-02,  1.44322202e-02,
        6.19336147e-03,  2.11727713e-03, -1.18888626e-03,  1.69878706e-03,
       -1.80248115e-03, 